In [1]:
from pathlib import Path
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
import seaborn as sns
sns.set_theme()
import math
import pandas as pd
import random
import numpy as np
import sys
import shutil

import IMP
import IMP.atom

sys.path.append(str(Path(Path.home(), "Documents/xray/src")))
from refine import refine_hs_to_max_ff

In [6]:
sample_df = pd.read_csv(Path(Path.home(), "Documents/xray/sample_bench/data/7mhf/187_bench_ref_10000/sample_per_out.csv"))
sample_df.head()

,Unnamed: 0,field,ff,pdb,r_free_0,w_0,N,job_id,out_id,r_free_1,w_1
0,0,6.079226,3263.960579,/wynton/group/sali/mhancock/xray/sample_bench/...,0.355274,1.0,1,0,171,NaN,NaN
1,1,6.097489,1983.084549,/wynton/group/sali/mhancock/xray/sample_bench/...,0.365177,1.0,1,0,1,NaN,NaN
2,2,6.103991,2120.197871,/wynton/group/sali/mhancock/xray/sample_bench/...,0.362373,1.0,1,0,18,NaN,NaN
3,3,6.089750,2065.275081,/wynton/group/sali/mhancock/xray/sample_bench/...,0.355195,1.0,1,0,23,NaN,NaN
4,4,6.085556,3244.989409,/wynton/group/sali/mhancock/xray/sample_bench/...,0.362210,1.0,1,0,20,NaN,NaN


In [7]:
sample_subset_df = sample_df[sample_df["job_id"] == 1]
best_row = sample_subset_df.loc[sample_subset_df["field"] == sample_subset_df["field"].min()]
print(best_row["pdb"].values[0])

/wynton/group/sali/mhancock/xray/sample_bench/out/7mhf/187_bench_ref_10000/1/output_23/pdbs/271.pdb


In [11]:
input_df.to_csv(str(input_file)+".test.csv")